In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision import datasets, transforms

`torchvision`의 `Image Transform` 에 대하여 생소하다면 **다음의 링크를 참고**해 주시기 바랍니다.

- [torchvision의 transform으로 이미지 정규화하기(평균, 표준편차를 계산하여 적용)](https://teddylee777.github.io/pytorch/torchvision-transform)
- [PyTorch 이미지 데이터셋(Image Dataset) 구성에 관한 거의 모든 것!](https://teddylee777.github.io/pytorch/dataset-dataloader)


`개와 고양이` 데이터셋을 다운로드 받아서 `tmp` 폴더에 압축을 풀어 줍니다.

[데이터셋 출처: 캐글, 마이크로소프트](https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip)


In [ ]:
import os
import requests

if not os.path.exists("competition.py"):
    url = "https://link.teddynote.com/COMPT"
    file_name = "competition.py"
    response = requests.get(url)
    with open(file_name, "wb") as file:
        file.write(response.content)

In [ ]:
import competition

# 파일 다운로드
competition.download_competition_files(
    f"https://link.teddynote.com/CATSDOGS", use_competition_url=False
)

하단의 `code snippets`는 corrupted 된 이미지를 확인하고 제거하기 위한 코드 입니다. `Cats vs Dogs`데이터셋에도 원인 모를 이유 때문에 이미지 데이터가 corrupt된 파일이 2개가 존재합니다. 이렇게 corrupt 된 이미지를 `DataLoader`로 로드시 에러가 발생하기 때문에 전처리 때 미리 제거하도록 하겠습니다.


`개와 고양이` 데이터셋을 시각화 하기 위하여 임시 `DataLoader`를 생성합니다.


In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


# 이미지 폴더로부터 데이터를 로드합니다.
dataset = ImageFolder(
    root="data/images",  # 다운로드 받은 폴더의 root 경로를 지정합니다.
    transform=transforms.Compose(
        [
            # 개와 고양이 사진 파일의 크기가 다르므로, Resize로 맞춰줍니다.
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]
    ),
)

data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8)

In [ ]:
# ImageFolder로부터 로드한 dataset의 클래스를 확인합니다.
# 총 2개의 클래스로 구성되었음을 확인할 수 있습니다(cats, dogs)
dataset.classes

In [ ]:
# 1개의 배치를 추출합니다.
images, labels = next(iter(data_loader))

In [ ]:
# 이미지의 shape을 확인합니다. 224 X 224 RGB 이미지 임을 확인합니다.
images[0].shape

`개와 고양이` 데이터셋 시각화

- 총 2개의 class(강아지/고양이)로 구성된 사진 파일입니다.


In [ ]:
import matplotlib.pyplot as plt
import torchvision

# 한 배치의 이미지 시각화 함수 (사이즈 조정 포함)


def imshow(img, labels, classes):
    img = img.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(20, 20))  # 여기에서 플롯의 크기를 조정
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    # 이미지마다 클래스 레이블을 타이틀로 표시
    for i, label in enumerate(labels):
        x = (i % 8) * (img.shape[1] / 8) + (img.shape[1] / 16)
        y = (i // 8) * (img.shape[0] / 4) + 10  # 4 rows
        plt.text(
            x,
            y,
            classes[label],
            ha="center",
            va="top",
            color="white",
            fontsize=12,
            backgroundcolor="black",
        )
    plt.show()


# 데이터 로더에서 한 배치 가져오기
dataiter = iter(data_loader)
images, labels = next(dataiter)

# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(images, nrow=8)  # 8개의 이미지를 한 줄에 표시

# 이미지와 레이블 시각화
imshow(img_grid, labels, dataset.classes)

## train / validation 데이터셋 split


현재 `cats and dogs`데이터셋에 하나의 데이터셋으로 구성된 Image 파일을 2개의 데이터셋(train/test)으로 분할하도록 하겠습니다.


## Image Augmentation 적용


`Image Augmentation`을 적용 합니다.

- [PyTorch Image Augmentation 도큐먼트](https://pytorch.org/vision/stable/transforms.html)


In [ ]:
# Image Transform을 지정합니다.
image_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # (224, 224) 이미지 크기 조정
        transforms.RandomHorizontalFlip(0.5),  # 50% 확률로 Horizontal Flip
        transforms.ToTensor(),  # Tensor 변환
        #     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), # 이미지 정규화
    ]
)

In [ ]:
# 이미지 폴더로부터 데이터를 로드합니다.
dataset = ImageFolder(
    root="data/images",  # 다운로드 받은 폴더의 root 경로를 지정합니다.
    transform=image_transform,
)  # Image Augmentation 적용

In [ ]:
# Image Augmentation 이 적용된 DataLoader를 로드 합니다.
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=8)

# 1개의 배치를 추출합니다.
images, labels = next(iter(data_loader))

In [ ]:
import torchvision

# 한 배치의 이미지 시각화 함수 (사이즈 조정 포함)


def imshow(img, labels, classes):
    img = img.numpy().transpose((1, 2, 0))
    plt.figure(figsize=(20, 20))  # 여기에서 플롯의 크기를 조정
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    # 이미지마다 클래스 레이블을 타이틀로 표시
    for i, label in enumerate(labels):
        x = (i % 8) * (img.shape[1] / 8) + (img.shape[1] / 16)
        y = (i // 8) * (img.shape[0] / 4) + 10  # 4 rows
        plt.text(
            x,
            y,
            classes[label],
            ha="center",
            va="top",
            color="white",
            fontsize=12,
            backgroundcolor="black",
        )
    plt.show()


# 데이터 로더에서 한 배치 가져오기
dataiter = iter(data_loader)
images, labels = next(dataiter)

# 이미지 그리드 만들기
img_grid = torchvision.utils.make_grid(images, nrow=8)  # 8개의 이미지를 한 줄에 표시

# 이미지와 레이블 시각화
imshow(img_grid, labels, dataset.classes)

In [ ]:
from torch.utils.data import random_split


ratio = 0.8  # 학습셋(train set)의 비율을 설정합니다.

train_size = int(ratio * len(dataset))
test_size = len(dataset) - train_size
print(f"total: {len(dataset)}\ntrain_size: {train_size}\ntest_size: {test_size}")

# random_split으로 8:2의 비율로 train / test 세트를 분할합니다.
train_data, test_data = random_split(dataset, [train_size, test_size])

## torch.utils.data.DataLoader

`DataLoader`는 배치 구성과 shuffle등을 편하게 구성해 주는 util 입니다.


In [ ]:
batch_size = 32  # batch_size 지정
num_workers = 8  # Thread 숫자 지정 (병렬 처리에 활용할 쓰레드 숫자 지정)

train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_loader = DataLoader(
    test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

`train_loader`의 1개 배치의 shape 출력


In [ ]:
images, labels = next(iter(train_loader))
images.shape, labels.shape

배치사이즈인 32가 가장 첫번째 dimension에 출력되고, 그 뒤로 채널(3), 세로(224px), 가로(224px) 순서로 출력이 됩니다.

즉, `224 X 224` RGB 컬러 이미지 `32장`이 1개의 배치로 구성이 되어 있습니다.


In [ ]:
# 1개의 이미지의 shape를 확인합니다.
# 224 X 224 RGB 이미지가 잘 로드 되었음을 확인합니다.
images[0].shape

In [ ]:
labels

## pre-trained 모델 로드


CUDA 설정이 되어 있다면 `cuda`를! 그렇지 않다면 `cpu`로 학습합니다.

(제 PC에는 GPU가 2대 있어서 `cuda:0`로 GPU 장비의 index를 지정해 주었습니다. 만약 다른 장비를 사용하고 싶다면 `cuda:1` 이런식으로 지정해 주면 됩니다)


In [ ]:
# CUDA 사용 가능 여부 확인
if torch.backends.mps.is_built():
    # mac os mps 지원 체크
    device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
else:
    # cuda 사용 가능한지 체크
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

**pre-trained model**을 fine tuning 하여 Image Classification을 구현하도록 하겠습니다.


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision import models  # pretrained 모델을 가져오기 위한 import

# VGG16 모델 생성
# weights=True 로 설정, weights=False로 설정되었을 경우 가중치는 가져오지 않습니다.
model = models.vgg16(weights=True)

그 밖의 활용 가능한 pretrained 모델

- `models.alexnet(pregrained=True)` # AlexNet
- `models.resnet18(pretrained=True)` # ResNet18
- `models.inception_v3(pretrained=True)` # Inception_V3


In [ ]:
# 가중치를 Freeze 하여 학습시 업데이트가 일어나지 않도록 설정합니다.
for param in model.parameters():
    param.requires_grad = False  # 가중치 Freeze

In [ ]:
# Fully-Connected Layer를 Sequential로 생성하여 VGG pretrained 모델의 'Classifier'에 연결합니다.
fc = nn.Sequential(
    # VGG16 모델의 features의 출력이 7X7, 512장 이기 때문에 in_features=7*7*512 로 설정합니다.
    nn.Linear(7 * 7 * 512, 256),
    nn.ReLU(),
    nn.Linear(256, 64),
    nn.ReLU(),
    nn.Linear(
        64, 2
    ),  # Cats vs Dogs 이진 분류이기 때문에 2로 out_features=2로 설정합니다.
)

In [ ]:
# Classifier에 FC layer 대입
model.classifier = fc

In [ ]:
# Classifier의 requires_grad 확인
for param in model.classifier.parameters():
    print(param.requires_grad)

In [ ]:
model.to(device)
# 모델의 구조도 출력
model

In [ ]:
# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
loss_fn = nn.CrossEntropyLoss()

## 훈련(Train) & 평가(Evaluate)


In [ ]:
from tqdm import tqdm


def fit(model, data_loader, loss_fn, optimizer, device, phase="train"):
    if phase == "train":
        # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
        model.train()
    else:
        # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다.
        model.eval()

    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0

    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, leave=False)

    # mini-batch 학습을 시작합니다.
    for img, lbl in prograss_bar:
        # image, label 데이터를 device에 올립니다.
        img, lbl = img.to(device), lbl.to(device)

        optimizer.zero_grad()
        # 누적 Gradient를 초기화 합니다.
        with torch.set_grad_enabled(phase == "train"):

            # Forward Propagation을 진행하여 결과를 얻습니다.
            output = model(img)

            # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
            loss = loss_fn(output, lbl)

            if phase == "train":
                # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
                loss.backward()

                # 계산된 Gradient를 업데이트 합니다.
                optimizer.step()

        # output 의 뉴런별 확률 값을 sparse vector 로 변환합니다.
        pred = output.argmax(axis=1)

        # 정답 개수를 카운트 합니다.
        corr += (lbl == pred).sum().item()

        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item()

    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)

    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader), acc

## 모델 훈련(training) & 검증


In [ ]:
import time

# 최대 Epoch을 지정합니다.
num_epochs = 5

min_loss = np.inf

STATE_DICT_PATH = "VGG-Pretrained-Model.pth"

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    start = time.time()
    train_loss, train_acc = fit(
        model, train_loader, loss_fn, optimizer, device, phase="train"
    )

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = fit(
        model, test_loader, loss_fn, optimizer, device, phase="eval"
    )

    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(
            f"[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!"
        )
        min_loss = val_loss
        torch.save(model.state_dict(), STATE_DICT_PATH)

    time_elapsed = time.time() - start
    # Epoch 별 결과를 출력합니다.
    print(
        f"[Epoch{epoch+1:02d}] time: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s \t loss: {train_loss:.5f}, acc: {train_acc:.5f} | val_loss: {val_loss:.5f}, val_acc: {val_acc:.5f}"
    )

## 저장한 가중치 로드 후 검증 성능 측정


In [ ]:
# 모델에 저장한 가중치를 로드합니다.
model.load_state_dict(torch.load(STATE_DICT_PATH))

In [ ]:
# 최종 검증 손실(validation loss)와 검증 정확도(validation accuracy)를 산출합니다.
final_loss, final_acc = fit(
    model, test_loader, loss_fn, optimizer, device, phase="eval"
)
print(
    f"\nevaluation loss: {final_loss:.5f}, evaluation accuracy: {final_acc:.5f}")